In [361]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
# import xmltojson
import json
import requests

In [362]:
def range_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lowend = soup.find_all("span", {"class":"value", "itemprop":"lowprice", "content": True})
    highend = soup.find_all("span", {"class":"value", "itemprop":"highprice", "content": True})
    quantityList = []
    rangePrice = ""

    for i in lowend:
        rangePrice+=(i["content"])
        rangePrice+=" - "
        break

    for i in highend:
        rangePrice+=(i["content"])
        break

    return rangePrice


In [363]:
def scrape_handguns_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    url+= "=undefined&start=0&sz=100"
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    ammo = soup.find_all("div", class_="pdp-link")
    ammoPrice = soup.find_all("div",  class_="tile-body")
    names = []
    prices = []
    actual_prices = []
    for i in ammo:
        names.append(i.find("span").text)

    for i in ammoPrice:
        # if the product has a range of prices go here
        if '<span class="range">' in str(i):
            x = i.find("a", {"class": "link", "href":True})
            prices.append(x["href"])

        # if the product has a slashed price, go here
        else:
            x = (i.find("span", class_="sales"))
            prices.append(x.find("span", {"class":"value", "content": True}))


    # add prices in the prices list
    for i in prices:
        if "http" in i:
            rp = range_price(i)
            actual_prices.append(rp)
            print(rp)

        else:
            htmlPriceTag = str(i)
            splitHtmlPriceTag = htmlPriceTag.split(" ")
            p =  splitHtmlPriceTag[2].strip('content="')
            print(p)
            actual_prices.append(str(p))


        # if str(type(i)) == "NoneType":
        #     continue

        # elif "http" in i:
        #     x = range_price(i)
        #     prices.append(x)

        # else:
        #     x = str(i)
        #     if " - " not in x:
        #         y = x.split(" ")
        #         try:
        #             z = y[2][9:]
        #             p = z.strip('"')
        #             prices.append(p)
        #         except:
        #             print("NOT DOABLE")
 

        #     # range prices go here
        #     else:
        #         prices.append(x)


            
    try:
        productList = pd.DataFrame(
            {"name": names, "price": actual_prices}
        )
        print(productList)

    except:
        print("something went wrong")

In [364]:
# https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/?https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/=undefined&start=0&sz=36
handgunByCaliberLink = "https://www.cheaperthandirt.com/shop-by?cgid=78&searchBy=Caliber"
def scrape_handguns(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    sections = soup.find_all("ul",class_= "col-lg-11 right-list")
    dirtyLinks = []  
    calibers = []  # names
    linkToCaliberList = []  # links
    
    for i in sections:
        dirtyLinks.append(i.find_all("a"))

    for i in dirtyLinks:
        for j in i:
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(j))
            linkToCaliberList.append(str(aLink[0]).replace('">', ""))
            calibers.append(j.find("span").text)


    handgunCaliberLinks = pd.DataFrame(
        {"caliber": calibers, "links": linkToCaliberList}
    )

    productList = []
    for l in handgunCaliberLinks["links"]:
        productList.append(scrape_handguns_helper(l))
        
    return productList

In [365]:
print(scrape_handguns(handgunByCaliberLink))

199.89
49.89
29.89
49.89
59.89
43.01
59.89
34.89
34.89
39.89
                                                name price
0  American Quality 10mm Auto Ammunition 250 Bulk...    []
1  Sellier & Bellot 10mm Auto Ammunition 50 Round...    []
2  Browning X-Point Defense 10mm Auto Ammunition ...    []
3  Magtech 10mm Auto Ammunition 50 Rounds FMJ 180...    []
4  Sellier & Bellot 10mm Ammunition 50 Rounds JHP...    []
5  Blazer Brass 10mm Auto 180 Grain FMJ Brass 120...    []
6  CCI Blazer 10mm Auto Ammunition 50 Rounds 200 ...    []
7  Buffalo Bore Tactical 10mm Auto Ammunition 20 ...    []
8  Buffalo Bore Tactical 10mm Auto Ammunition 20 ...    []
9  Buffalo Bore Tactical 10mm Auto Ammunition 20 ...    []
49.89
42.83
                                                name price
0  Winchester USA .25 ACP Ammunition 50 Rounds, F...    []
1  Sellier & Bellot .25 ACP Ammunition 50 Rounds ...    []
34.89
                                                name price
0  Buffalo Bore .327 Federal Ammunit

In [366]:
# aString = ["CTDhandgun", "CTDrifle", "CTDshotgun"]
# num = 0
# for i in AmmoList:
#     res = i.to_json(orient="split")
#     parsed = json.loads(res)
#     file = json.dumps(parsed, indent=2)
#     with open(f"{aString[num]}.json", "w") as fp:
#         json.dump(file, fp)
#     num += 1






# savePath = "/Users/chakaneshegog/Desktop/Ammo/ammo_finder/ammo_finder/jsonData/cheaperThanDirt"
# import os, shutil, glob
# try:
#     os.makedirs(savePath)  # creates a destination for directory
# except OSError:
#     pass

# for data in glob.iglob("*.json"):
#     shutil.copy2(data, savePath)